In [4]:
import numpy as np

In [6]:
def read_input(name = 'input.txt'): 
    with open(name, "r") as f:
        n = int(f.readline().strip())
        a = np.zeros((n, n), dtype=float)
        b = np.zeros(n, dtype=float)
        for i in range(n):
            a[i] = np.fromstring(f.readline(), sep=" ")
        b = np.fromstring(f.readline(), sep=" ")
    
    return a, b

In [69]:
def write_output(x, true_x, name = "output.txt"):
    try:
        abs_err = np.sqrt(np.sum((x - true_x) ** 2))
        rel_err = abs_err / np.sqrt(np.sum(true_x ** 2))
    
        with open(name, "w") as f:
            f.write("My Result:\t" + np.array2string(x, precision=9, separator=', '))
            f.write("\nNumPy Result: " + np.array2string(true_x, precision=9, separator=', ') + "\n")
            f.write(f"Absolute error: {abs_err:.6e}\n")
            f.write(f"Relative error: {rel_err:.6e}\n")
    except:
        with open(name, "w") as f:
            f.write("Matrix failed to converge")
            return

In [87]:
def seidel(A, b, tol=1e-10): 
    A = np.array(A, dtype=np.float64)
    b = np.array(b, dtype=np.float64)
    n = len(b)
    x0 = np.zeros(n, dtype=np.float64)
    x = np.array(x0, dtype=np.float64)
    
    diag = np.abs(np.diag(A))
    off_diag_sum = np.sum(np.abs(A), axis=1) - diag
    if np.all(diag > off_diag_sum):
        print("Validation: Matrix is strictly diagonally dominant. Convergence is guaranteed.")
    else:
        print("Validation Warning: Matrix is NOT strictly diagonally dominant.")
        print("                Convergence is NOT guaranteed (method may diverge).")
        
    # Оцінка 'q'
    q = np.max(off_diag_sum / diag)
    if q < 1:
        # Використовуємо "надійний" критерій: ((1-q)/q) * epsilon
        stop_tol = ((1 - q) / q) * tol
    else:
        # Використовуємо "простий" критерій, оскільки формула недійсна
        stop_tol = tol
        
    if A.shape[0] != n or A.shape[1] != n:
        print(f"Error: Matrix A shape {A.shape} is not compatible with vector b length {n}.")
        return None

    k = 0
    while True: 
        k += 1
        x_old = x.copy()

        for i in range(n):
            if A[i, i] == 0:
                print(f"Error: Zero diagonal element A[{i}][{i}]. Method fails.")
                return None
                
            sum1 = np.dot(A[i, :i], x[:i])
            sum2 = np.dot(A[i, i+1:], x_old[i+1:])
            x[i] = (b[i] - sum1 - sum2) / A[i, i]

        
        diff_norm = np.max(np.abs(x - x_old))
        
        #Умова зупинки тепер використовує 'stop_tol'
        if diff_norm < stop_tol:
            print(f"\nConvergence reached after {k} iterations.")
            return x


        if not np.isfinite(diff_norm) or diff_norm > 1e100:
            print(f"\nProcess stopped after {k} iterations: Method is diverging.")
            return None


In [98]:
def test_seidel(n, tol=1e-10):
    x_true = np.arange(1, n + 1, dtype=np.float64)
    A = np.random.rand(n, n) * 10 - 5
    
    off_diagonal_sum = np.sum(np.abs(A), axis=1) - np.abs(np.diag(A))
    np.fill_diagonal(A, off_diagonal_sum + n)

    b = A @ x_true
    
    seidel_solution = seidel(A, b, tol)

    return seidel_solution, x_true

In [100]:
A, B = read_input("input1.txt")
write_output(seidel(A, B), np.linalg.solve(A, B), "solution1.txt")

Validation Warning: Matrix is NOT strictly diagonally dominant.
                Convergence is NOT guaranteed (method may diverge).

Convergence reached after 574 iterations.


In [102]:
A, B = read_input("input2.txt")
write_output(seidel(A, B), np.linalg.solve(A, B), "solution2.txt")

Validation Warning: Matrix is NOT strictly diagonally dominant.
                Convergence is NOT guaranteed (method may diverge).

Process stopped after 158 iterations: Method is diverging.


In [104]:
solution, real_solution = test_seidel(1000)
print(f"Absolute error: {np.sqrt(np.sum((solution - real_solution) ** 2))}")
print(f"Relative error: {np.sqrt(np.sum((solution - real_solution) ** 2)) / np.sqrt(np.sum(real_solution ** 2))}")

Validation: Matrix is strictly diagonally dominant. Convergence is guaranteed.

Convergence reached after 9 iterations.
Absolute error: 4.10770349836492e-12
Relative error: 2.2481957889590993e-16
